# HW3 : predict the stock movement
## dataset
   [2009/1~2019/1](https://www.sharecast.com/index/SP_500/prices/download)


In [169]:
import pandas as pd

path = './download.csv'
pd_data = pd.read_csv(path)
pd_data.head()

import warnings
warnings.filterwarnings("ignore")


## Preprocessing
### create the column of stock movement
    和前一天比，increase:1, decrease:0

In [154]:
pd_data['movement'] = 0

for i in range(0, 2515):
    if(pd_data['Close Price'][i] < pd_data['Close Price'][i+1]):
        pd_data['movement'][i] = 1
    
pd_data.head()


,Date,Open Price,Close Price,High Price,Low Price,Volume,movement
0,02-Jan-2009,902.99,931.80,934.73,899.35,4048270080,0
1,05-Jan-2009,929.17,927.45,936.63,919.53,5413910016,1
2,06-Jan-2009,931.17,934.70,943.85,927.28,5392620032,0
3,07-Jan-2009,927.45,906.65,927.45,902.37,4704940032,1
4,08-Jan-2009,905.73,909.73,910.00,896.81,4991549952,0


### split train set and test set
    Attribute：Open Price（開盤價）、Close Price（收盤價）、High Price（當日最高價）、Low Price（當日最低價）、Volume（當日成交量）
    前2264筆資料當作train set，2009~2017年
    剩餘252筆資料當作test set，2018年
    

In [155]:
train = pd_data[:2264]
test = pd_data[2264:]

#train set x
xtr = train[['Open Price', 'Close Price', 'High Price', 'Low Price', 'Volume']]
#train set y
ytr = train[['movement']]
#test set x 
xte = test[['Open Price', 'Close Price', 'High Price', 'Low Price', 'Volume']]
#test set y
yte = test[['movement']]

print(xtr.head())
print(ytr.head())

   Open Price  Close Price  High Price  Low Price      Volume
0      902.99       931.80      934.73     899.35  4048270080
1      929.17       927.45      936.63     919.53  5413910016
2      931.17       934.70      943.85     927.28  5392620032
3      927.45       906.65      927.45     902.37  4704940032
4      905.73       909.73      910.00     896.81  4991549952
   movement
0         0
1         1
2         0
3         1
4         0


### standardization

In [156]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(xtr)
xtr_std = scaler.transform(xtr)
xte_std = scaler.transform(xte)

print(xtr_std)

[[-1.55230001 -1.49441197 -1.50545617 -1.54093327  0.81362581]
 [-1.49834121 -1.50337884 -1.50153623 -1.49936521  1.82418053]
 [-1.49421907 -1.48843406 -1.48664047 -1.48340126  1.80842623]
 ...
 [ 2.1145693   2.11464105  2.10688987  2.1247189  -1.33171387]
 [ 2.12281357  2.12478289  2.11105738  2.13250518 -1.34874724]
 [ 2.12909984  2.09606828  2.12025892  2.11380162 -1.19609981]]


## 1.Logistic Regression

### 用預設的參數
    準確率不太好，只有0.52

In [160]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

LR = LogisticRegression()
LR.fit(xtr_std, ytr)

result = LR.predict(xte_std)
acc_rate = accuracy_score(yte, result)
print(acc_rate)
print(result)

0.5198412698412699
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1]


### 把volume拿掉試試看
    準確率不變，作用不大

In [161]:
tr_std = xtr_std[:, 0:4]
te_std = xte_std[:, 0:4]

LR.fit(tr_std, ytr)
result = LR.predict(te_std)
acc_rate = accuracy_score(yte, result)
print(acc_rate)

0.5198412698412699


## 2.SVM
### 用預設的參數，發現準確率偏低
    default: penalty = 1, kernel = rbf
    準確率仍為0.52

In [163]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(xtr_std, ytr)

result = svm.predict(xte_std)
acc_rate = accuracy_score(yte, result)
print(acc_rate)
print(result)

0.5198412698412699
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


### 換個kernel試試看
    當kernel = linear，準確律還是不變
    accuracy: 0.52

In [166]:
svm = SVC(kernel = 'linear')
svm.fit(xtr_std, ytr)

result = svm.predict(xte_std)
acc_rate = accuracy_score(yte, result)
print(acc_rate)
print(result)

0.5198412698412699
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


### 調高penalty
    測試過後，最佳的penalty是9
    準確值意外的來到0.61

In [164]:
svm = SVC(C = 9)
svm.fit(xtr_std, ytr)

result = svm.predict(xte_std)
acc_rate = accuracy_score(yte, result)
print(acc_rate)
print(result)

0.6111111111111112
[1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


## Neural Network


## Another dataset: iris data set

In [126]:
from sklearn import datasets
iris = datasets.load_iris()

print(iris.feature_names)
print(iris.data[:5])
print(iris.target_names)
print(iris.target[:5])
#iris['data']
#iris['target']

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
['setosa' 'versicolor' 'virginica']
[0 0 0 0 0]


### Preprocessing

In [123]:
#split train set and test set
from sklearn.model_selection import train_test_split
xtr, xte, ytr, yte = train_test_split(iris['data'], iris['target'])

#standardlization
scaler.fit(xtr)
xtr_std = scaler.transform(xtr)
xte_std = scaler.transform(xte)

print(xtr_std[:5])

[[-1.19665371  0.69777693 -1.12455785 -1.2145154 ]
 [-0.82641831 -1.37092644 -0.32559695 -0.03623674]
 [ 0.0374643  -0.91121458  0.30215804 -0.16715659]
 [-1.44347731  0.23806507 -1.23869512 -1.2145154 ]
 [ 1.39499411  0.00820914  0.8728444   1.53480146]]


### Logistic Regression、SVM、Neural Network

In [125]:
#Logistic Regression 
LR = LogisticRegression()
LR.fit(xtr_std, ytr)
result = LR.predict(xte_std)
acc_rate = accuracy_score(yte, result)
print("logistic regression: %.4f" % (acc_rate))

#SVM
svm = SVC()
svm.fit(xtr_std, ytr)
result = svm.predict(xte_std)
acc_rate = accuracy_score(yte, result)
print("SVM: %.4f" % (acc_rate))

#Neural Network

logistic regression: 0.9474
SVM: 0.9211
